In [26]:
!pip install keras
!pip install tensorflow

In [28]:
from keras.applications.vgg16 import VGG16
from keras.layers import Flatten, Dense
from keras.models import Model
import cv2

# Load the VGGFace model
model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='avg')

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.save("model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [35]:
import cv2
import numpy as np
from keras.models import load_model
from keras.applications.vgg16 import preprocess_input  # Import preprocess_input
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import euclidean
# Load the saved VGG16 model
model = load_model("model.h5")

# Rest of your code...

def extract_vggface_features(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB format
    img = cv2.resize(img, (224, 224))
    img = preprocess_input(img.astype(np.float32))

    img = np.expand_dims(img, axis=0)  # Add batch dimension
    features = model.predict(img)
    return features

def extract_orb_features(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    orb = cv2.ORB_create()
    _, descriptors = orb.detectAndCompute(img, None)
    return descriptors


In [50]:
# Define a list to store base image features and labels
base_image_features = []
base_image_labels = [1,2,3,4]

# Assuming you have 4 base images for 4 classes
base_image_paths = ["base1.jpg", "base2.jpg", "base3.jpg", "base4.jpg"]
for path in base_image_paths:
    vgg_features = extract_vggface_features(path)
    orb_features = extract_orb_features(path)
    # Append features and labels to the lists
    base_image_features.append((vgg_features, orb_features))

1/1 [==============================] - 0s 363ms/step


In [51]:
# Extract features from the test image
test_image_path = "base1.jpg"
test_vgg_features = extract_vggface_features(test_image_path)
test_orb_features = extract_orb_features(test_image_path)

test_image_features=[]
test_image_features.append((test_vgg_features,test_orb_features))
# Create a list to store similarity scores between test and base images



1/1 [==============================] - 0s 362ms/step


In [52]:


# Define a function to calculate similarity
def calculate_similarity(test_features, base_features):
    # Flatten the arrays if they are not already flattened
    test_vgg_flat = test_features[0].flatten() if len(test_features[0].shape) > 1 else test_features[0]
    test_orb_flat = test_features[1].flatten() if len(test_features[1].shape) > 1 else test_features[1]
    base_vgg_flat = base_features[0].flatten() if len(base_features[0].shape) > 1 else base_features[0]
    base_orb_flat = base_features[1].flatten() if len(base_features[1].shape) > 1 else base_features[1]

    # Combine the flattened arrays with padding or truncation
    max_length = max(len(test_vgg_flat), len(test_orb_flat), len(base_vgg_flat), len(base_orb_flat))

    test_vgg_flat = np.pad(test_vgg_flat, (0, max_length - len(test_vgg_flat)))
    test_orb_flat = np.pad(test_orb_flat, (0, max_length - len(test_orb_flat)))
    base_vgg_flat = np.pad(base_vgg_flat, (0, max_length - len(base_vgg_flat)))
    base_orb_flat = np.pad(base_orb_flat, (0, max_length - len(base_orb_flat)))

    # Calculate Euclidean distance on the combined features
    combined_similarity = euclidean(np.concatenate((test_vgg_flat, test_orb_flat)),
                                     np.concatenate((base_vgg_flat, base_orb_flat)))

    return combined_similarity

# ...
similarities=[]
# Loop through each base image and calculate similarity
for i, base_features in enumerate(base_image_features):
    similarity_score = calculate_similarity((test_vgg_features, test_orb_features), base_features)
    similarities.append((similarity_score, i))

# Find the index of the base image with the minimum similarity score
if similarities:
    closest_base_index = min(similarities, key=lambda x: x[0])[1]
    assigned_class_label = base_image_labels[closest_base_index]
    print(f"The assigned class label is: {assigned_class_label}")
else:
    print("No similarity scores calculated because 'similarities' list is empty.")


The assigned class label is: 1
